Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [12]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import timeit
from functools import reduce

In [3]:
pickle_file = '../1_notmnist/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- ***convolutions need the image data formatted as a cube (width by height by #channels)***
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [45]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #The stride of the sliding window for each dimension of input [1, 2, 2, 1]
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [46]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.551120
Minibatch accuracy: 12.5%
Validation accuracy: 11.6%
Minibatch loss at step 50: 1.070435
Minibatch accuracy: 62.5%
Validation accuracy: 57.6%
Minibatch loss at step 100: 0.672634
Minibatch accuracy: 81.2%
Validation accuracy: 70.0%
Minibatch loss at step 150: 0.729239
Minibatch accuracy: 75.0%
Validation accuracy: 77.1%
Minibatch loss at step 200: 0.751937
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 250: 0.361574
Minibatch accuracy: 93.8%
Validation accuracy: 79.1%
Minibatch loss at step 300: 1.045674
Minibatch accuracy: 68.8%
Validation accuracy: 79.9%
Minibatch loss at step 350: 0.858527
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.986527
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.719866
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.551902
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
M

Validation accuracy: 87.2%
Minibatch loss at step 4550: 0.491586
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 4600: 0.243106
Minibatch accuracy: 93.8%
Validation accuracy: 87.6%
Minibatch loss at step 4650: 0.502938
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 4700: 0.954889
Minibatch accuracy: 62.5%
Validation accuracy: 87.5%
Minibatch loss at step 4750: 0.894277
Minibatch accuracy: 75.0%
Validation accuracy: 87.2%
Minibatch loss at step 4800: 0.335516
Minibatch accuracy: 93.8%
Validation accuracy: 87.3%
Minibatch loss at step 4850: 0.417851
Minibatch accuracy: 75.0%
Validation accuracy: 87.9%
Minibatch loss at step 4900: 0.478776
Minibatch accuracy: 81.2%
Validation accuracy: 87.4%
Minibatch loss at step 4950: 0.507516
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 5000: 0.070713
Minibatch accuracy: 100.0%
Validation accuracy: 87.7%
Minibatch loss at step 5050: 0.677399
Minibatch accuracy: 75.0

Validation accuracy: 88.0%
Minibatch loss at step 9050: 0.626671
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 9100: 0.099821
Minibatch accuracy: 100.0%
Validation accuracy: 88.9%
Minibatch loss at step 9150: 0.004823
Minibatch accuracy: 100.0%
Validation accuracy: 89.0%
Minibatch loss at step 9200: 0.528787
Minibatch accuracy: 81.2%
Validation accuracy: 88.6%
Minibatch loss at step 9250: 0.197111
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 9300: 0.724096
Minibatch accuracy: 75.0%
Validation accuracy: 88.3%
Minibatch loss at step 9350: 0.164430
Minibatch accuracy: 100.0%
Validation accuracy: 89.2%
Minibatch loss at step 9400: 0.005159
Minibatch accuracy: 100.0%
Validation accuracy: 88.7%
Minibatch loss at step 9450: 1.254859
Minibatch accuracy: 81.2%
Validation accuracy: 88.6%
Minibatch loss at step 9500: 0.371030
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 9550: 0.217543
Minibatch accuracy: 9

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

### Solution 1

In [47]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    
    pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    bias2 = tf.nn.relu(conv2 + layer2_biases)
    
    pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [49]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.181582
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.835237
Minibatch accuracy: 25.0%
Validation accuracy: 20.1%
Minibatch loss at step 100: 0.986038
Minibatch accuracy: 62.5%
Validation accuracy: 53.2%
Minibatch loss at step 150: 1.137571
Minibatch accuracy: 75.0%
Validation accuracy: 71.1%
Minibatch loss at step 200: 1.047514
Minibatch accuracy: 68.8%
Validation accuracy: 75.5%
Minibatch loss at step 250: 0.351182
Minibatch accuracy: 93.8%
Validation accuracy: 78.7%
Minibatch loss at step 300: 1.043608
Minibatch accuracy: 68.8%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.611621
Minibatch accuracy: 68.8%
Validation accuracy: 79.7%
Minibatch loss at step 400: 1.114435
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.448903
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 500: 0.595472
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
Mi

Validation accuracy: 88.0%
Minibatch loss at step 4550: 0.380746
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 4600: 0.207142
Minibatch accuracy: 93.8%
Validation accuracy: 88.0%
Minibatch loss at step 4650: 0.654989
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 4700: 0.931857
Minibatch accuracy: 62.5%
Validation accuracy: 88.1%
Minibatch loss at step 4750: 0.712597
Minibatch accuracy: 81.2%
Validation accuracy: 88.0%
Minibatch loss at step 4800: 0.403200
Minibatch accuracy: 81.2%
Validation accuracy: 87.9%
Minibatch loss at step 4850: 0.236645
Minibatch accuracy: 100.0%
Validation accuracy: 88.3%
Minibatch loss at step 4900: 0.277449
Minibatch accuracy: 93.8%
Validation accuracy: 88.2%
Minibatch loss at step 4950: 0.345839
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 5000: 0.150099
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 5050: 0.725721
Minibatch accuracy: 75.0

Validation accuracy: 88.7%
Minibatch loss at step 9100: 0.113254
Minibatch accuracy: 100.0%
Validation accuracy: 89.4%
Minibatch loss at step 9150: 0.003980
Minibatch accuracy: 100.0%
Validation accuracy: 89.1%
Minibatch loss at step 9200: 0.475099
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 9250: 0.215411
Minibatch accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss at step 9300: 0.560960
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 9350: 0.170800
Minibatch accuracy: 93.8%
Validation accuracy: 89.5%
Minibatch loss at step 9400: 0.014281
Minibatch accuracy: 100.0%
Validation accuracy: 89.4%
Minibatch loss at step 9450: 1.354242
Minibatch accuracy: 62.5%
Validation accuracy: 89.0%
Minibatch loss at step 9500: 0.170745
Minibatch accuracy: 100.0%
Validation accuracy: 88.8%
Minibatch loss at step 9550: 0.194515
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 9600: 0.722358
Minibatch accuracy: 8

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### Solution 2

In [43]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # C1 input 28 x 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 24 x 24
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 12 x 12
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 8 x 8
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F6 input 4 x 4
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [44]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.526541
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.414705
Minibatch accuracy: 56.2%
Validation accuracy: 43.8%
Minibatch loss at step 100: 1.124609
Minibatch accuracy: 56.2%
Validation accuracy: 61.4%
Minibatch loss at step 150: 1.353289
Minibatch accuracy: 68.8%
Validation accuracy: 71.2%
Minibatch loss at step 200: 0.947133
Minibatch accuracy: 81.2%
Validation accuracy: 73.4%
Minibatch loss at step 250: 0.517881
Minibatch accuracy: 87.5%
Validation accuracy: 75.5%
Minibatch loss at step 300: 0.956672
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
Minibatch loss at step 350: 0.723126
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 400: 1.075581
Minibatch accuracy: 75.0%
Validation accuracy: 76.4%
Minibatch loss at step 450: 0.550407
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 500: 0.726005
Minibatch accuracy: 68.8%
Validation accuracy: 78.4%
Mi

Minibatch loss at step 4550: 0.670770
Minibatch accuracy: 81.2%
Validation accuracy: 87.2%
Minibatch loss at step 4600: 0.275050
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 4650: 0.696253
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 4700: 0.936110
Minibatch accuracy: 68.8%
Validation accuracy: 87.2%
Minibatch loss at step 4750: 0.860129
Minibatch accuracy: 75.0%
Validation accuracy: 87.6%
Minibatch loss at step 4800: 0.490172
Minibatch accuracy: 87.5%
Validation accuracy: 87.1%
Minibatch loss at step 4850: 0.437755
Minibatch accuracy: 81.2%
Validation accuracy: 87.4%
Minibatch loss at step 4900: 0.440230
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 4950: 0.488433
Minibatch accuracy: 81.2%
Validation accuracy: 87.3%
Minibatch loss at step 5000: 0.056312
Minibatch accuracy: 100.0%
Validation accuracy: 87.4%
Minibatch loss at step 5050: 0.726131
Minibatch accuracy: 75.0%
Validation accuracy: 87.7

Minibatch loss at step 9050: 0.498141
Minibatch accuracy: 75.0%
Validation accuracy: 88.0%
Minibatch loss at step 9100: 0.248634
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 9150: 0.006733
Minibatch accuracy: 100.0%
Validation accuracy: 88.5%
Minibatch loss at step 9200: 0.634819
Minibatch accuracy: 81.2%
Validation accuracy: 88.3%
Minibatch loss at step 9250: 0.301908
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 9300: 0.663130
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 9350: 0.145026
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 9400: 0.064727
Minibatch accuracy: 100.0%
Validation accuracy: 88.6%
Minibatch loss at step 9450: 1.379012
Minibatch accuracy: 68.8%
Validation accuracy: 88.6%
Minibatch loss at step 9500: 0.325227
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Minibatch loss at step 9550: 0.179339
Minibatch accuracy: 93.8%
Validation accuracy: 88.

#### Dropout and learning rate decay 3-layer CNN

In [51]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta_regul = 1e-3
drop_out = 0.5

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob):
    # C1 input 28 x 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 24 x 24
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 12 x 12
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 8 x 8
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F5 input 4 x 4
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    # F6
    drop5 = tf.nn.dropout(hidden5, keep_prob)
    hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_biases)
    drop6 = tf.nn.dropout(hidden6, keep_prob)
    return tf.matmul(drop6, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset, drop_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [53]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.437034
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.098214
Minibatch accuracy: 25.0%
Validation accuracy: 25.6%
Minibatch loss at step 100: 1.369801
Minibatch accuracy: 50.0%
Validation accuracy: 52.5%
Minibatch loss at step 150: 1.442206
Minibatch accuracy: 37.5%
Validation accuracy: 61.0%
Minibatch loss at step 200: 0.943950
Minibatch accuracy: 62.5%
Validation accuracy: 59.0%
Minibatch loss at step 250: 0.893753
Minibatch accuracy: 68.8%
Validation accuracy: 62.3%
Minibatch loss at step 300: 1.322172
Minibatch accuracy: 62.5%
Validation accuracy: 69.2%
Minibatch loss at step 350: 1.163786
Minibatch accuracy: 68.8%
Validation accuracy: 73.5%
Minibatch loss at step 400: 1.102786
Minibatch accuracy: 62.5%
Validation accuracy: 74.6%
Minibatch loss at step 450: 0.774450
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss at step 500: 0.703000
Minibatch accuracy: 75.0%
Validation accuracy: 76.4%
Mi

Minibatch loss at step 4550: 0.522259
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 4600: 0.689430
Minibatch accuracy: 75.0%
Validation accuracy: 86.5%
Minibatch loss at step 4650: 0.411999
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 4700: 1.076067
Minibatch accuracy: 62.5%
Validation accuracy: 86.1%
Minibatch loss at step 4750: 0.965814
Minibatch accuracy: 75.0%
Validation accuracy: 86.3%
Minibatch loss at step 4800: 0.628654
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 4850: 0.434307
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 4900: 0.490865
Minibatch accuracy: 75.0%
Validation accuracy: 86.7%
Minibatch loss at step 4950: 0.597011
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 5000: 0.196026
Minibatch accuracy: 93.8%
Validation accuracy: 86.4%
Minibatch loss at step 5050: 0.729391
Minibatch accuracy: 75.0%
Validation accuracy: 86.3%

Minibatch loss at step 9050: 0.926771
Minibatch accuracy: 81.2%
Validation accuracy: 87.5%
Minibatch loss at step 9100: 0.338009
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 9150: 0.032997
Minibatch accuracy: 100.0%
Validation accuracy: 87.3%
Minibatch loss at step 9200: 0.542197
Minibatch accuracy: 81.2%
Validation accuracy: 87.5%
Minibatch loss at step 9250: 0.468681
Minibatch accuracy: 87.5%
Validation accuracy: 87.6%
Minibatch loss at step 9300: 0.882163
Minibatch accuracy: 75.0%
Validation accuracy: 87.4%
Minibatch loss at step 9350: 0.123082
Minibatch accuracy: 100.0%
Validation accuracy: 87.3%
Minibatch loss at step 9400: 0.035082
Minibatch accuracy: 100.0%
Validation accuracy: 87.6%
Minibatch loss at step 9450: 1.789538
Minibatch accuracy: 75.0%
Validation accuracy: 87.4%
Minibatch loss at step 9500: 0.329209
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 9550: 0.233760
Minibatch accuracy: 93.8%
Validation accuracy: 87

#### Another example

In [7]:
# Graph with GPU support

batch_size = 32
patch_size = 5

g = tf.Graph()

config = tf.ConfigProto(allow_soft_placement = True)

with tf.device('/gpu:0'):
#with g.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.1, global_step, 10000, 0.95, staircase=True)
    
    # Variables.
    W_1 = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, 6],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_1 = tf.Variable(tf.zeros([6]))
    
    W_2 = tf.get_variable("W2", shape=[patch_size, patch_size, 6, 16],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_2 = tf.Variable(tf.zeros([16]))
    
    W_3 = tf.get_variable("W3", shape=[5 * 5 * 16, 120],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_3 = tf.Variable(tf.zeros([120]))
    
    W_4 = tf.get_variable("W4", shape=[120, 84],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_4 = tf.Variable(tf.zeros([84]))
    
    W_5 = tf.get_variable("W5", shape=[84, num_labels],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_5 = tf.Variable(tf.zeros([num_labels]))
    
    # Model.
    def model(data, dropout=False):
        conv = tf.nn.conv2d(data, W_1, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + b_1)
        
        pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        conv = tf.nn.conv2d(pool, W_2, [1, 1, 1, 1], padding='VALID')
        hidden = tf.nn.relu(conv + b_2)
        
        pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])

        hidden = tf.nn.relu(tf.matmul(reshape, W_3) + b_3)
        if dropout:
            hidden = tf.nn.dropout(hidden, 0.5)

        hidden = tf.nn.relu(tf.matmul(hidden, W_4) + b_4)
        if dropout:
            hidden = tf.nn.dropout(hidden, 0.5)
        
        return tf.matmul(hidden, W_5) + b_5
    
    # Training computation.
    logits = model(tf_train_dataset, dropout=True)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
      
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [13]:
# Optimization

n_epochs = 200 # number of epochs

# early-stopping parameters
patience = 5000 # look as this many examples regardless
patience_increase = 2
improvement_threshold = 0.995 # a relative improvement of 
                                  # this much is considered significant
best_valid_loss = np.inf
start_time = timeit.default_timer()
prev_time = start_time

n_train_batches = train_dataset.shape[0] // batch_size

valid_freq = min(n_train_batches, patience // 2)

epoch_freq_ratio = max(1, n_train_batches // (patience // 2))

delta_t = []
with tf.Session(config=config) as sess:
#with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    done_looping = False
    epoch = 0
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):
            
            batch_data = \
                train_dataset[minibatch_index * batch_size:(minibatch_index + 1) * batch_size, :]
            batch_labels = \
                train_labels[minibatch_index * batch_size:(minibatch_index + 1) * batch_size, :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = sess.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
            iter = (epoch - 1) * n_train_batches + minibatch_index # cumulative iteration number
        
            if (iter + 1) % valid_freq == 0:
                this_valid_loss = 1. - (accuracy(valid_prediction.eval(), valid_labels) / 100.)
            
                print("Minibatch loss at epoch %i and iter %i: %f and the learning rate: %f" % 
                      (epoch, iter, l, learning_rate.eval()))
                print("Minibatch  train, validation accuracy: %.3f%%, %.3f%%" 
                    % (accuracy(predictions, batch_labels), accuracy(valid_prediction.eval(), valid_labels)))

                end_time = timeit.default_timer()

                delta_t.append(end_time - prev_time)
                
                print('Time interval: %.4f seconds, estimated run time for %i epochs: %.4f hours' % 
                        ((end_time - prev_time), n_epochs, 
                        ((reduce(lambda x, y: x + y, delta_t) / len(delta_t)) 
                         * n_epochs * epoch_freq_ratio) / 3600.))
                
                prev_time = end_time
            
                if this_valid_loss < best_valid_loss:
                    if this_valid_loss < best_valid_loss * improvement_threshold:
                        patience = max(patience, iter * patience_increase)
                    
                    best_valid_loss = this_valid_loss
                    
                    params = (sess.run(W_1), sess.run(b_1),
                              sess.run(W_2), sess.run(b_2),
                              sess.run(W_3), sess.run(b_3),
                              sess.run(W_4), sess.run(b_4),
                              sess.run(W_5), sess.run(b_5))
                    
                    # save the best model
                    with open('best_model_params.pkl', 'wb') as f:
                            pickle.dump(params, f)
                    print('Model saved')
        
            if patience <= iter:
                    done_looping = True
                    break
                    
    print("Final Test accuracy: %.1f%%" 
                  % accuracy(test_prediction.eval(), test_labels))
    
    with open('best_model_params.pkl', 'rb') as f:
        (W_1_best, b_1_best, 
         W_2_best, b_2_best,
         W_3_best, b_3_best, 
         W_4_best, b_4_best,
         W_5_best, b_5_best) = pickle.load(f)
    
    W_1_init, b_1_init = tf.assign(W_1, W_1_best), tf.assign(b_1, b_1_best)
    W_2_init, b_2_init = tf.assign(W_2, W_2_best), tf.assign(b_2, b_2_best)
    W_3_init, b_3_init = tf.assign(W_3, W_3_best), tf.assign(b_3, b_3_best)
    W_4_init, b_4_init = tf.assign(W_4, W_4_best), tf.assign(b_4, b_4_best)
    W_5_init, b_5_init = tf.assign(W_5, W_5_best), tf.assign(b_5, b_5_best)
    
    sess.run([W_1_init, b_1_init,
              W_2_init, b_2_init,
              W_3_init, b_3_init,
              W_4_init, b_4_init,
              W_5_init, b_5_init])
    
    print("Test accuracy with the best model: %.3f%%" 
                  % accuracy(test_prediction.eval(), test_labels))

    end_time = timeit.default_timer()

    print('Total run time %.4f minutes' % ((end_time - start_time) / 60.))

    sess.close()

Initialized
Minibatch loss at epoch 1 and iter 2499: 0.384034 and the learning rate: 0.100000
Minibatch  train, validation accuracy: 93.750%, 85.850%
Time interval: 5.5342 seconds, estimated run time for 200 epochs: 0.6149 hours
Model saved
Minibatch loss at epoch 1 and iter 4999: 0.668850 and the learning rate: 0.100000
Minibatch  train, validation accuracy: 78.125%, 87.110%
Time interval: 5.0998 seconds, estimated run time for 200 epochs: 0.5908 hours
Model saved
Minibatch loss at epoch 2 and iter 7499: 1.115576 and the learning rate: 0.100000
Minibatch  train, validation accuracy: 68.750%, 88.480%
Time interval: 5.0702 seconds, estimated run time for 200 epochs: 0.5816 hours
Model saved
Minibatch loss at epoch 2 and iter 9999: 0.426022 and the learning rate: 0.095000
Minibatch  train, validation accuracy: 84.375%, 88.750%
Time interval: 5.1273 seconds, estimated run time for 200 epochs: 0.5787 hours
Model saved
Minibatch loss at epoch 2 and iter 12499: 0.674072 and the learning rate

Minibatch loss at epoch 16 and iter 94999: 0.888272 and the learning rate: 0.063025
Minibatch  train, validation accuracy: 65.625%, 91.100%
Time interval: 5.1978 seconds, estimated run time for 200 epochs: 0.5766 hours
Minibatch loss at epoch 16 and iter 97499: 0.403756 and the learning rate: 0.063025
Minibatch  train, validation accuracy: 90.625%, 91.250%
Time interval: 5.1449 seconds, estimated run time for 200 epochs: 0.5765 hours
Minibatch loss at epoch 16 and iter 99999: 0.490652 and the learning rate: 0.059874
Minibatch  train, validation accuracy: 84.375%, 91.240%
Time interval: 5.1756 seconds, estimated run time for 200 epochs: 0.5764 hours
Minibatch loss at epoch 17 and iter 102499: 0.132998 and the learning rate: 0.059874
Minibatch  train, validation accuracy: 96.875%, 91.150%
Time interval: 5.1194 seconds, estimated run time for 200 epochs: 0.5762 hours
Minibatch loss at epoch 17 and iter 104999: 0.420577 and the learning rate: 0.059874
Minibatch  train, validation accuracy:

Minibatch loss at epoch 30 and iter 187499: 0.438321 and the learning rate: 0.039721
Minibatch  train, validation accuracy: 84.375%, 91.580%
Time interval: 5.2438 seconds, estimated run time for 200 epochs: 0.5747 hours
Minibatch loss at epoch 31 and iter 189999: 0.086360 and the learning rate: 0.037735
Minibatch  train, validation accuracy: 96.875%, 91.630%
Time interval: 5.0907 seconds, estimated run time for 200 epochs: 0.5746 hours
Minibatch loss at epoch 31 and iter 192499: 0.351889 and the learning rate: 0.037735
Minibatch  train, validation accuracy: 87.500%, 91.730%
Time interval: 5.0764 seconds, estimated run time for 200 epochs: 0.5744 hours
Model saved
Minibatch loss at epoch 32 and iter 194999: 1.031806 and the learning rate: 0.037735
Minibatch  train, validation accuracy: 71.875%, 91.680%
Time interval: 5.0897 seconds, estimated run time for 200 epochs: 0.5743 hours
Minibatch loss at epoch 32 and iter 197499: 0.287940 and the learning rate: 0.037735
Minibatch  train, valid

Minibatch loss at epoch 46 and iter 282499: 0.936623 and the learning rate: 0.023783
Minibatch  train, validation accuracy: 68.750%, 91.830%
Time interval: 5.7227 seconds, estimated run time for 200 epochs: 0.5724 hours
Minibatch loss at epoch 46 and iter 284999: 0.276604 and the learning rate: 0.023783
Minibatch  train, validation accuracy: 90.625%, 91.920%
Time interval: 5.2061 seconds, estimated run time for 200 epochs: 0.5725 hours
Model saved
Minibatch loss at epoch 46 and iter 287499: 0.347641 and the learning rate: 0.023783
Minibatch  train, validation accuracy: 87.500%, 91.730%
Time interval: 5.5605 seconds, estimated run time for 200 epochs: 0.5729 hours
Minibatch loss at epoch 47 and iter 289999: 0.042983 and the learning rate: 0.022594
Minibatch  train, validation accuracy: 100.000%, 91.790%
Time interval: 5.1909 seconds, estimated run time for 200 epochs: 0.5729 hours
Minibatch loss at epoch 47 and iter 292499: 0.263486 and the learning rate: 0.022594
Minibatch  train, vali

Minibatch loss at epoch 61 and iter 377499: 0.039296 and the learning rate: 0.014989
Minibatch  train, validation accuracy: 100.000%, 91.860%
Time interval: 5.1723 seconds, estimated run time for 200 epochs: 0.5796 hours
Minibatch loss at epoch 61 and iter 379999: 0.299408 and the learning rate: 0.014240
Minibatch  train, validation accuracy: 90.625%, 91.900%
Time interval: 5.0669 seconds, estimated run time for 200 epochs: 0.5795 hours
Minibatch loss at epoch 62 and iter 382499: 0.890123 and the learning rate: 0.014240
Minibatch  train, validation accuracy: 75.000%, 91.930%
Time interval: 5.0796 seconds, estimated run time for 200 epochs: 0.5794 hours
Minibatch loss at epoch 62 and iter 384999: 0.301274 and the learning rate: 0.014240
Minibatch  train, validation accuracy: 87.500%, 92.110%
Time interval: 5.0836 seconds, estimated run time for 200 epochs: 0.5793 hours
Model saved
Minibatch loss at epoch 62 and iter 387499: 0.538551 and the learning rate: 0.014240
Minibatch  train, vali

Minibatch loss at epoch 76 and iter 472499: 0.250756 and the learning rate: 0.008974
Minibatch  train, validation accuracy: 87.500%, 92.030%
Time interval: 5.3149 seconds, estimated run time for 200 epochs: 0.5785 hours
Minibatch loss at epoch 76 and iter 474999: 0.409959 and the learning rate: 0.008974
Minibatch  train, validation accuracy: 87.500%, 92.000%
Time interval: 5.1973 seconds, estimated run time for 200 epochs: 0.5785 hours
Minibatch loss at epoch 77 and iter 477499: 0.104257 and the learning rate: 0.008974
Minibatch  train, validation accuracy: 96.875%, 92.040%
Time interval: 5.3220 seconds, estimated run time for 200 epochs: 0.5786 hours
Minibatch loss at epoch 77 and iter 479999: 0.229731 and the learning rate: 0.008526
Minibatch  train, validation accuracy: 93.750%, 92.100%
Time interval: 5.1914 seconds, estimated run time for 200 epochs: 0.5786 hours
Minibatch loss at epoch 78 and iter 482499: 0.680556 and the learning rate: 0.008526
Minibatch  train, validation accura

Minibatch loss at epoch 91 and iter 567499: 0.279891 and the learning rate: 0.005656
Minibatch  train, validation accuracy: 90.625%, 92.040%
Time interval: 4.8428 seconds, estimated run time for 200 epochs: 0.5773 hours
Minibatch loss at epoch 92 and iter 569999: 0.750679 and the learning rate: 0.005373
Minibatch  train, validation accuracy: 78.125%, 92.080%
Time interval: 5.1424 seconds, estimated run time for 200 epochs: 0.5773 hours
Minibatch loss at epoch 92 and iter 572499: 0.178353 and the learning rate: 0.005373
Minibatch  train, validation accuracy: 90.625%, 92.030%
Time interval: 5.1905 seconds, estimated run time for 200 epochs: 0.5773 hours
Minibatch loss at epoch 92 and iter 574999: 0.402424 and the learning rate: 0.005373
Minibatch  train, validation accuracy: 84.375%, 92.130%
Time interval: 4.9400 seconds, estimated run time for 200 epochs: 0.5772 hours
Minibatch loss at epoch 93 and iter 577499: 0.044976 and the learning rate: 0.005373
Minibatch  train, validation accura

Minibatch loss at epoch 106 and iter 662499: 0.335319 and the learning rate: 0.003387
Minibatch  train, validation accuracy: 90.625%, 92.050%
Time interval: 4.9607 seconds, estimated run time for 200 epochs: 0.5760 hours
Minibatch loss at epoch 107 and iter 664999: 0.058613 and the learning rate: 0.003387
Minibatch  train, validation accuracy: 100.000%, 92.110%
Time interval: 4.9624 seconds, estimated run time for 200 epochs: 0.5759 hours
Minibatch loss at epoch 107 and iter 667499: 0.208186 and the learning rate: 0.003387
Minibatch  train, validation accuracy: 93.750%, 92.120%
Time interval: 5.0932 seconds, estimated run time for 200 epochs: 0.5759 hours
Minibatch loss at epoch 108 and iter 669999: 0.705385 and the learning rate: 0.003217
Minibatch  train, validation accuracy: 75.000%, 92.070%
Time interval: 5.0231 seconds, estimated run time for 200 epochs: 0.5758 hours
Minibatch loss at epoch 108 and iter 672499: 0.294402 and the learning rate: 0.003217
Minibatch  train, validation 

Minibatch loss at epoch 121 and iter 754999: 0.236434 and the learning rate: 0.002134
Minibatch  train, validation accuracy: 93.750%, 92.160%
Time interval: 5.1287 seconds, estimated run time for 200 epochs: 0.5745 hours
Minibatch loss at epoch 122 and iter 757499: 0.705483 and the learning rate: 0.002134
Minibatch  train, validation accuracy: 78.125%, 92.070%
Time interval: 5.6939 seconds, estimated run time for 200 epochs: 0.5747 hours
Minibatch loss at epoch 122 and iter 759999: 0.204496 and the learning rate: 0.002028
Minibatch  train, validation accuracy: 93.750%, 92.090%
Time interval: 4.9165 seconds, estimated run time for 200 epochs: 0.5746 hours
Minibatch loss at epoch 122 and iter 762499: 0.383367 and the learning rate: 0.002028
Minibatch  train, validation accuracy: 87.500%, 92.050%
Time interval: 4.8903 seconds, estimated run time for 200 epochs: 0.5745 hours
Minibatch loss at epoch 123 and iter 764999: 0.061040 and the learning rate: 0.002028
Minibatch  train, validation a